### Pre-Selection

In [5]:
import pandas as pd
import src.functions as src

features = "dataset1_features.csv"
features = pd.read_csv(features, sep=',', header=0, index_col=None)
Y = features[['labels']]
# feat_not_theta = [i for i in features.columns if "Theta" not in i]
# X = features.drop(feat_not_theta, axis=1)
# not_time_feat = [i for i in features.columns if ("ERN" not in i) & ("Pe" not in i) & ("amp" not in i)]
# X = features.drop(not_time_feat, axis=1)

#not_freq_feat = [i for i in features.columns if ("Delta" not in i) & ("Theta" not in i) & ("Alpha" not in i) &
 #                ("Freq" not in i)]
#X = features.drop(not_freq_feat, axis=1)
# X = features.drop('Participant', axis=1)
X = features.drop('labels', axis=1)
print(len(X.columns)) 

1629


### Feature Selection

In [6]:
from mrmr import mrmr_classif

selected_features = mrmr_classif(X=X, y=Y, K=35)
combined_df=pd.DataFrame()
for i in selected_features:
    combined_df=pd.concat([combined_df, X[i]], axis=1)
X=combined_df

100%|██████████| 35/35 [00:12<00:00,  2.75it/s]


### Classification

In [7]:
src.classification(X,Y)

Mean Sensitivity: 0.3226
Std Sensitivity: 0.0638
Mean Specificity: 0.9251
Std Specificity: 0.0113
Mean Balanced Accuracy: 0.6239
Std Balanced Accuracy: 0.0304
